In [1]:
!pip install hyperopt

In [15]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/dw_matrix3/dw_matrix3_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/dw_matrix3/dw_matrix3_road_sign


In [0]:
train =  pd.read_pickle('data/train.p')
test =  pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,  histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
 )
  return model

def predict (model_trained, X_test, y_test, scroing=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scroing(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
# adding this layer cause error Negative dimension size caused by subtracting 2 from 1 
# for '{{node max_pooling2d_257/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", ksize=[1, 2, 2, 1], 
# padding="VALID", strides=[1, 2, 2, 1]](conv2d_527/Identity)' with input shapes: [?,1,1,128]. 
  MaxPool2D(),
  Dropout(0.3),
  
  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')
    
  ])


In [52]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train )

predict(model_trained, X_test, y_test)


Epoch 1/5
272/272 [==============================] - 7s 24ms/step - loss: 2.3318 - accuracy: 0.3329 - val_loss: 0.7091 - val_accuracy: 0.7739
Epoch 2/5
272/272 [==============================] - 6s 23ms/step - loss: 0.5829 - accuracy: 0.8155 - val_loss: 0.1569 - val_accuracy: 0.9525
Epoch 3/5
272/272 [==============================] - 6s 23ms/step - loss: 0.2607 - accuracy: 0.9236 - val_loss: 0.0691 - val_accuracy: 0.9812
Epoch 4/5
272/272 [==============================] - 6s 23ms/step - loss: 0.1762 - accuracy: 0.9496 - val_loss: 0.0253 - val_accuracy: 0.9939
Epoch 5/5
272/272 [==============================] - 6s 23ms/step - loss: 0.1186 - accuracy: 0.9674 - val_loss: 0.0397 - val_accuracy: 0.9890


0.972108843537415

In [48]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 4ms/step - loss: 0.1095 - accuracy: 0.9741


0.9741496443748474

In [0]:
#train_and_predict(get_cnn_v5(input_shape, num_classes))

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['droput_cnn_block_one']),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['droput_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['droput_cnn_block_three']),
  
  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(params['droput_dense_block_one']),

  Dense(1024, activation='relu'),
  Dropout(params['droput_dense_block_two']),

  Dense(num_classes, activation='softmax')
    
  ])


In [0]:
def func_obj(params):
#  model = get_cnn_v5(input_shape, num_classes) first version to cheeck functioning
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 7,
      verbose = 0
  )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status':STATUS_OK, 'model': model}


In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'droput_cnn_block_one': hp.uniform('droput_cnn_block_one', 0.3, 0.5),
    'droput_cnn_block_two': hp.uniform('droput_cnn_block_two', 0.3, 0.5),
    'droput_cnn_block_three': hp.uniform('droput_cnn_block_three', 0.3, 0.5),
    'droput_dense_block_one': hp.uniform('droput_dense_block_one', 0.3, 0.7),
    'droput_dense_block_two': hp.uniform('droput_dense_block_two', 0.3, 0.7)
}

In [56]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 110.0, 'droput_cnn_block_one': 0.38524135581102453, 'droput_cnn_block_three': 0.3598298338006467, 'droput_cnn_block_two': 0.4323884194589918, 'droput_dense_block_one': 0.6393171885646163, 'droput_dense_block_two': 0.6305773776105661}
accuracy=0.942630410194397
{'batch_size': 180.0, 'droput_cnn_block_one': 0.35742384875759353, 'droput_cnn_block_three': 0.39471642541332663, 'droput_cnn_block_two': 0.41442190140895846, 'droput_dense_block_one': 0.5664988250213974, 'droput_dense_block_two': 0.5212180015246398}
accuracy=0.9580498933792114
{'batch_size': 170.0, 'droput_cnn_block_one': 0.39245913477210465, 'droput_cnn_block_three': 0.46168200879962323, 'droput_cnn_block_two': 0.3143703062775416, 'droput_dense_block_one': 0.6687785162142332, 'droput_dense_block_two': 0.5942553231879595}
accuracy=0.9539682269096375
{'batch_size': 150.0, 'droput_cnn_block_one': 0.34505289823314417, 'droput_cnn_block_three': 0.4574853331580323, 'droput_cnn_block_two': 0.37213367699183475, 'droput_d